# Recommandation avec cold start

In [1]:
import pandas as pd

import seaborn as sns

from deep_implicit_feedback_recsys import *

from cold_star_recommender import *

from recommender_evaluation import *

Using TensorFlow backend.


In [2]:
def normalized_results(df, recommender_type):
    df['average_reward_normalized_max'] = df['average_reward'] / df['max_price']
    df['average_reward_normalized_mean'] = df['average_reward'] / df['mean_price']
    df['recommendation'] = recommender_type
    return df

In [3]:
n_loop = 10
n_pred = 1000
online_batch_size = n_pred // 10

user_dim = item_dim = 8
n_hidden = 1
hidden_size = 64
dropout = 0.1
l2_reg = 0

ID = 'MC15CHD1JU24INEKKPU4'


## Mesures d'échantillon

### Implicit Recommender avec nuls sans indicateur, en ligne


In [4]:
%%time
ImplicitRecommenderWithNull_no_indicator_online_df = \
    eval_loop_ColdStartImplicitRecommender(ColdStartRecommenderClass=ColdStartImplicitRecommender,
                                           ImplicitRecommenderClass=ImplicitRecommenderWithNull_no_indicator,
                                           n_loop=n_loop, n_pred=n_pred,
                                           online_batch_size=1,
                                           user_dim=user_dim, item_dim=item_dim,
                                           n_hidden=n_hidden, hidden_size=hidden_size, 
                                           dropout=dropout, l2_reg=l2_reg,
                                           ID=ID)


CPU times: user 1min 9s, sys: 3.01 s, total: 1min 12s
Wall time: 2min 59s


In [5]:
ImplicitRecommenderWithNull_no_indicator_online_df_normalized = normalized_results(ImplicitRecommenderWithNull_no_indicator_online_df,
                                                                                   'Null, en ligne')
ImplicitRecommenderWithNull_no_indicator_online_df_normalized

,max_price,mean_price,average_reward,good_reco_ratio,average_reward_normalized_max,average_reward_normalized_mean,recommendation
0,998.766720,75.756518,49.746852,0.303,0.049808,0.656668,"Null, en ligne"
1,983.468961,73.556454,112.684132,0.265,0.114578,1.531941,"Null, en ligne"
2,995.357790,69.160918,31.421910,0.207,0.031568,0.454330,"Null, en ligne"
3,986.666723,69.725068,232.267220,0.258,0.235406,3.331187,"Null, en ligne"
4,985.774399,77.573272,42.517487,0.260,0.043131,0.548095,"Null, en ligne"
5,959.835162,78.252181,264.686221,0.330,0.275762,3.382477,"Null, en ligne"
6,995.640481,71.732071,38.528802,0.170,0.038698,0.537121,"Null, en ligne"
7,974.998852,77.153231,237.368186,0.262,0.243455,3.076581,"Null, en ligne"
8,968.734480,86.844925,146.698936,0.316,0.151434,1.689206,"Null, en ligne"
9,989.309151,73.150462,57.089341,0.349,0.057706,0.780437,"Null, en ligne"


### Implicit Recommender avec nuls et indicateur special, hors ligne

In [6]:
%%time
ImplicitRecommenderWithNull_df = \
    eval_loop_ColdStartImplicitRecommender(ColdStartRecommenderClass=ColdStartImplicitRecommender,
                                           ImplicitRecommenderClass=ImplicitRecommenderWithNull,
                                           n_loop=n_loop, n_pred=n_pred,
                                           user_dim=user_dim, item_dim=item_dim,
                                           n_hidden=n_hidden, hidden_size=hidden_size, 
                                           dropout=dropout, l2_reg=l2_reg,
                                          ID=ID)
                                           

CPU times: user 1min 20s, sys: 3.28 s, total: 1min 24s
Wall time: 2min 30s


In [7]:
ImplicitRecommenderWithNull_df_normalized = normalized_results(ImplicitRecommenderWithNull_df, 'Null special, hors ligne')
ImplicitRecommenderWithNull_df_normalized

,max_price,mean_price,average_reward,good_reco_ratio,average_reward_normalized_max,average_reward_normalized_mean,recommendation
0,901.145252,62.594552,253.000857,0.310,0.280755,4.041899,"Null special, hors ligne"
1,990.294390,72.306540,75.410592,0.256,0.076150,1.042929,"Null special, hors ligne"
2,997.964168,63.940242,300.659933,0.333,0.301273,4.702202,"Null special, hors ligne"
3,944.880876,79.529538,281.516886,0.318,0.297939,3.539778,"Null special, hors ligne"
4,926.679678,68.657573,203.784341,0.230,0.219908,2.968126,"Null special, hors ligne"
5,959.046139,71.107734,244.112128,0.280,0.254536,3.432990,"Null special, hors ligne"
6,963.041317,71.929299,249.186837,0.288,0.258750,3.464330,"Null special, hors ligne"
7,989.145228,66.413055,236.687906,0.269,0.239285,3.563876,"Null special, hors ligne"
8,868.631139,55.644972,234.748966,0.292,0.270252,4.218691,"Null special, hors ligne"
9,957.180313,69.877584,307.837332,0.344,0.321609,4.405380,"Null special, hors ligne"


### Implicit Recommender avec nuls et indicateur special, en ligne


In [8]:
%%time
ImplicitRecommenderWithNull_online_df = \
    eval_loop_ColdStartImplicitRecommender(ColdStartRecommenderClass=ColdStartImplicitRecommender,
                                           ImplicitRecommenderClass=ImplicitRecommenderWithNull,
                                           n_loop=n_loop, n_pred=n_pred,
                                           online_batch_size=1,
                                           user_dim=user_dim, item_dim=item_dim,
                                           n_hidden=n_hidden, hidden_size=hidden_size, 
                                           dropout=dropout, l2_reg=l2_reg,
                                          ID=ID)


CPU times: user 1min 15s, sys: 2.6 s, total: 1min 17s
Wall time: 2min 52s


In [9]:
ImplicitRecommenderWithNull_online_df_normalized = normalized_results(ImplicitRecommenderWithNull_online_df, 'Null special, en ligne')
ImplicitRecommenderWithNull_online_df_normalized

,max_price,mean_price,average_reward,good_reco_ratio,average_reward_normalized_max,average_reward_normalized_mean,recommendation
0,999.299521,77.830844,248.918116,0.272,0.249093,3.198194,"Null special, en ligne"
1,936.936738,70.032355,164.161711,0.204,0.175211,2.344084,"Null special, en ligne"
2,994.550219,77.347262,200.416841,0.227,0.201515,2.591130,"Null special, en ligne"
3,928.858915,70.318720,131.321117,0.288,0.141379,1.867513,"Null special, en ligne"
4,971.605797,74.729137,215.717802,0.255,0.222022,2.886663,"Null special, en ligne"
5,939.797371,62.063720,182.205905,0.218,0.193878,2.935788,"Null special, en ligne"
6,976.724118,67.924391,173.469683,0.200,0.177604,2.553864,"Null special, en ligne"
7,963.950999,68.021683,281.534816,0.326,0.292063,4.138898,"Null special, en ligne"
8,978.435624,60.751654,247.782304,0.281,0.253243,4.078610,"Null special, en ligne"
9,985.307590,79.129033,236.508105,0.271,0.240035,2.988892,"Null special, en ligne"


### Implicit Recommender avec nuls et indicateur special, en ligne version batch


In [10]:
%%time
ImplicitRecommenderWithNull_online_batch_df = \
    eval_loop_ColdStartImplicitRecommender(ColdStartRecommenderClass=ColdStartImplicitRecommender,
                                           ImplicitRecommenderClass=ImplicitRecommenderWithNull,
                                           n_loop=n_loop, n_pred=n_pred,
                                           online_batch_size=online_batch_size,
                                           user_dim=user_dim, item_dim=item_dim,
                                           n_hidden=n_hidden, hidden_size=hidden_size, 
                                           dropout=dropout, l2_reg=l2_reg,
                                          ID=ID)


CPU times: user 1min 18s, sys: 2.57 s, total: 1min 21s
Wall time: 3min 11s


In [11]:
ImplicitRecommenderWithNull_online_batch_df_normalized = \
    normalized_results(ImplicitRecommenderWithNull_online_batch_df, 'Null special, en ligne (batch)')
ImplicitRecommenderWithNull_online_batch_df_normalized

,max_price,mean_price,average_reward,good_reco_ratio,average_reward_normalized_max,average_reward_normalized_mean,recommendation
0,993.294422,65.180971,144.969472,0.173,0.145948,2.224107,"Null special, en ligne (batch)"
1,994.553663,73.187970,166.543434,0.196,0.167455,2.275557,"Null special, en ligne (batch)"
2,995.631048,82.826838,60.388889,0.290,0.060654,0.729098,"Null special, en ligne (batch)"
3,978.899954,85.014612,210.856980,0.234,0.215402,2.480244,"Null special, en ligne (batch)"
4,990.744124,80.640130,301.433052,0.316,0.304249,3.738003,"Null special, en ligne (batch)"
5,996.208164,81.434712,221.049558,0.254,0.221891,2.714439,"Null special, en ligne (batch)"
6,999.043292,76.149746,44.815510,0.301,0.044858,0.588518,"Null special, en ligne (batch)"
7,840.932895,51.871352,164.287793,0.225,0.195364,3.167216,"Null special, en ligne (batch)"
8,984.905034,73.500377,216.806642,0.243,0.220129,2.949735,"Null special, en ligne (batch)"
9,981.453383,56.314058,226.854080,0.271,0.231141,4.028374,"Null special, en ligne (batch)"


### Fully Implicit Recommender avec nuls et indicateur binaire, en ligne version batch


In [12]:
%%time
FullyImplicitRecommenderWithNull_binary_indicator_online_batch_df = \
    eval_loop_ColdStartImplicitRecommender(ColdStartRecommenderClass=ColdStartFullyImplicitRecommender,
                                           ImplicitRecommenderClass=ImplicitRecommenderWithNull_binary_indicator,
                                           n_loop=n_loop, n_pred=n_pred,
                                           online_batch_size=online_batch_size,
                                           user_dim=user_dim, item_dim=item_dim,
                                           n_hidden=n_hidden, hidden_size=hidden_size, 
                                           dropout=dropout, l2_reg=l2_reg,
                                          ID=ID)


CPU times: user 1min 4s, sys: 1.1 s, total: 1min 5s
Wall time: 2min 19s


In [13]:
FullyImplicitRecommenderWithNull_binary_indicator_online_batch_df_normalized = \
    normalized_results(FullyImplicitRecommenderWithNull_binary_indicator_online_batch_df, 'Fully Implicit Null binaire, en ligne (batch)')
FullyImplicitRecommenderWithNull_binary_indicator_online_batch_df_normalized

,max_price,mean_price,average_reward,good_reco_ratio,average_reward_normalized_max,average_reward_normalized_mean,recommendation
0,990.283354,69.465614,184.801905,0.301,0.186615,2.660336,"Fully Implicit Null binaire, en ligne (batch)"
1,988.195040,74.477663,213.000247,0.313,0.215545,2.859921,"Fully Implicit Null binaire, en ligne (batch)"
2,973.116413,71.898450,220.037979,0.333,0.226117,3.060399,"Fully Implicit Null binaire, en ligne (batch)"
3,991.012619,67.227482,146.913396,0.253,0.148246,2.185318,"Fully Implicit Null binaire, en ligne (batch)"
4,994.010691,61.481862,165.893161,0.315,0.166893,2.698246,"Fully Implicit Null binaire, en ligne (batch)"
5,987.764989,72.892346,177.164527,0.275,0.179359,2.430496,"Fully Implicit Null binaire, en ligne (batch)"
6,980.905391,64.996159,161.403226,0.319,0.164545,2.483273,"Fully Implicit Null binaire, en ligne (batch)"
7,943.194232,64.974689,206.668785,0.336,0.219116,3.180758,"Fully Implicit Null binaire, en ligne (batch)"
8,981.904634,65.251615,167.938706,0.299,0.171034,2.573710,"Fully Implicit Null binaire, en ligne (batch)"
9,994.941235,59.326771,82.001419,0.251,0.082418,1.382199,"Fully Implicit Null binaire, en ligne (batch)"


## Aggregation des résutats

In [14]:
all_results = pd.concat( (
    ImplicitRecommenderWithNull_no_indicator_online_df_normalized,
    ImplicitRecommenderWithNull_df_normalized,
    ImplicitRecommenderWithNull_online_df_normalized,
    ImplicitRecommenderWithNull_online_batch_df_normalized,
    
    FullyImplicitRecommenderWithNull_binary_indicator_online_batch_df_normalized,
), axis = 0)

In [15]:
aggregat = all_results.groupby('recommendation').mean()

### Classement ratio de recommandations effectives

In [16]:
col = 'good_reco_ratio' 
aggregat.sort_values(col, ascending=False)\
    .style.apply((lambda x: ['background: lightgreen' if x.name == col else '' for _ in x]))

,max_price,mean_price,average_reward,good_reco_ratio,average_reward_normalized_max,average_reward_normalized_mean
recommendation,,,,,,
"Fully Implicit Null binaire, en ligne (batch)",982.533,67.1993,172.582,0.2995,0.175989,2.55147
"Null special, hors ligne",949.801,68.2001,238.695,0.292,0.252046,3.53802
"Null, en ligne",983.855,75.2905,121.301,0.272,0.124155,1.5988
"Null special, en ligne",967.547,70.8149,208.204,0.2542,0.214604,2.95836
"Null special, en ligne (batch)",975.567,72.6121,175.801,0.2503,0.180709,2.48953


### Classement par average reward normalisée (max)

In [17]:
col = 'average_reward_normalized_max' 
aggregat.sort_values(col, ascending=False)\
    .style.apply((lambda x: ['background: lightgreen' if x.name == col else '' for _ in x]))

,max_price,mean_price,average_reward,good_reco_ratio,average_reward_normalized_max,average_reward_normalized_mean
recommendation,,,,,,
"Null special, hors ligne",949.801,68.2001,238.695,0.292,0.252046,3.53802
"Null special, en ligne",967.547,70.8149,208.204,0.2542,0.214604,2.95836
"Null special, en ligne (batch)",975.567,72.6121,175.801,0.2503,0.180709,2.48953
"Fully Implicit Null binaire, en ligne (batch)",982.533,67.1993,172.582,0.2995,0.175989,2.55147
"Null, en ligne",983.855,75.2905,121.301,0.272,0.124155,1.5988


### Classement par average reward normalisée (mean)

In [18]:
col = 'average_reward_normalized_mean' 
aggregat.sort_values(col, ascending=False)\
    .style.apply((lambda x: ['background: lightgreen' if x.name == col else '' for _ in x]))

,max_price,mean_price,average_reward,good_reco_ratio,average_reward_normalized_max,average_reward_normalized_mean
recommendation,,,,,,
"Null special, hors ligne",949.801,68.2001,238.695,0.292,0.252046,3.53802
"Null special, en ligne",967.547,70.8149,208.204,0.2542,0.214604,2.95836
"Fully Implicit Null binaire, en ligne (batch)",982.533,67.1993,172.582,0.2995,0.175989,2.55147
"Null special, en ligne (batch)",975.567,72.6121,175.801,0.2503,0.180709,2.48953
"Null, en ligne",983.855,75.2905,121.301,0.272,0.124155,1.5988


### Résumé (classé par average_reward)

In [19]:
cm = sns.light_palette("green", as_cmap=True)
col = ['average_reward_normalized_max', 'average_reward_normalized_mean']
aggregat.sort_values(col, ascending=False)\
    .style.background_gradient(cmap=cm).highlight_max(color='red')

,max_price,mean_price,average_reward,good_reco_ratio,average_reward_normalized_max,average_reward_normalized_mean
recommendation,,,,,,
"Null special, hors ligne",949.801,68.2001,238.695,0.292,0.252046,3.53802
"Null special, en ligne",967.547,70.8149,208.204,0.2542,0.214604,2.95836
"Null special, en ligne (batch)",975.567,72.6121,175.801,0.2503,0.180709,2.48953
"Fully Implicit Null binaire, en ligne (batch)",982.533,67.1993,172.582,0.2995,0.175989,2.55147
"Null, en ligne",983.855,75.2905,121.301,0.272,0.124155,1.5988


La recommendation prenant en charge les nulls semble meilleure. <br>
  1) Null secial en ligne (batch) <br>
  2) Fully Implicit Null special, en ligne (batch) <br>
  3) Null, hors ligne <br>